## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_lookup = spark.read.csv("/user/sprenner/lookup_mai.csv", sep="\t")

In [5]:
#df_lookup.describe().toPandas()

In [6]:
#df_lookup.limit(10).toPandas()

In [7]:
df_lookup_renamed = df_lookup.toDF('SCOPE', 'NAME', 'CHILD_SCOPE', 'CHILD_NAME', 'DID_TYPE', 'CHILD_TYPE', 
                                   'LENGTH', 'BYTES', 'ADLER32', 'MD5', 'RULE_EVALUATION', 'UPDATED_AT', 
                                   'CREATED_AT', 'EVENTS')

In [8]:
df_mai = spark.read.json("/user/sprenner/training_official_mai.json")

In [9]:
df_mai = df_mai.withColumn("join_key_mai", concat_ws(':', df_mai["scope"], df_mai["filename"]))
df_mai = df_mai.withColumn("fileScope", df_mai["scope"])
df_mai = df_mai.drop("scope")
df_mai.createOrReplaceTempView("mai")
df_lookup_renamed = df_lookup_renamed.withColumn("join_key_lookup", concat_ws(':', df_lookup_renamed["CHILD_SCOPE"], df_lookup_renamed["CHILD_NAME"]))
df_lookup_renamed.createOrReplaceTempView("lookup_renamed")
df_lookup_renamed.count()

831654692

In [10]:
df_lookup_reduced = spark.sql("SELECT *\
                                 FROM lookup_renamed\
                                 WHERE SCOPE != 'panda' AND DID_TYPE == 'D' AND (NAME LIKE 'data%' OR NAME LIKE 'mc%')")
df_lookup_reduced.count()
# without official: 789897421

624370615

In [11]:
#df_joined_check_6 = df_joined_check_6.drop('LENGTH','ADLER32','MD5','RULE_EVALUATION','EVENTS')

In [12]:
df_lookup_reduced.createOrReplaceTempView("lookup_reduced")

dataset_count = spark.sql("SELECT *\
                      FROM (SELECT join_key_lookup AS join_key_lookup_count, COUNT(DISTINCT NAME) AS distinct_datasets\
                      FROM lookup_reduced \
                      GROUP BY join_key_lookup \
                      ORDER BY distinct_datasets DESC)")
dataset_count.limit(20).toPandas()
# oberstes: data10_7TeV:data10_7TeV.00152220.physics_L1CaloEM.merge.RAW._lb0002._0001.1     4

join_key_lookup_count  \
0   data10_cvalid:data10_7TeV.00152994.debug_EFHltTimeout.daq.RAW._lb0000._SFO-3._0001.data    
1   data10_7TeV:data10_7TeV.00152214.physics_L1Calo.merge.RAW._lb0171._0001.1                  
2   data10_7TeV:data10_7TeV.00152508.physics_MinBias.merge.RAW._lb0191._0006.1                 
3   data10_7TeV:data10_7TeV.00152220.physics_L1CaloEM.merge.RAW._lb0002._0001.1                
4   data17_13TeV:data17_13TeV.00339849.physics_EnhancedBias.merge.RAW._lb0173._SFO-4._0001.1   
5   data10_cvalid:data10_7TeV.00152878.physics_RNDM.merge.RAW._lb0008._0001.1                  
6   data10_cvalid:data10_7TeV.00152844.physics_MuonswBeam.merge.RAW._lb0167._0001.1            
7   data11_7TeV:data11_7TeV.00178021.physics_Muons.merge.RAW._lb0014._SFO-4._0001.1            
8   data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0261._0006.1                  
9   data09_900GeV:data09_900GeV.00141398.physics_BPTX.merge.RAW._lb0012._0004.1                
10  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0336._0007.1                  
11  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0314._0002.1                  
12  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0270._0005.1                  
13  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0302._0006.1                  
14  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0317._0002.1                  
15  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0270._0001.1                  
16  data09_900GeV:data09_900GeV.00140571.physics_MuonswBeam.merge.RAW._lb0034._0001.1          
17  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0250._0003.1                  
18  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0266._0001.1                  
19  data09_900GeV:data09_900GeV.00141561.physics_MuonswBeam.merge.RAW._lb0051._0001.1          

    distinct_datasets  
0   4                  
1   4                  
2   4                  
3   4                  
4   4                  
5   4                  
6   4                  
7   3                  
8   3                  
9   3                  
10  3                  
11  3                  
12  3                  
13  3                  
14  3                  
15  3                  
16  3                  
17  3                  
18  3                  
19  3

In [13]:
dataset_count.count()

620848088

In [14]:
dataset_count.createOrReplaceTempView("dataset_count")
df_lookup_dataset_count = spark.sql("SELECT * \
                                     FROM lookup_reduced JOIN dataset_count ON lookup_reduced.join_key_lookup == dataset_count.join_key_lookup_count")
df_lookup_dataset_count.limit(10).toPandas()

SCOPE  \
0  data08_1beammag   
1  data08_1beammag   
2  data08_1beammag   
3  data08_1beammag   
4  data08_1beammag   
5  data08_1beammag   
6  data08_1beammag   
7  data08_1beammag   
8  data08_1beammag   
9  data08_1beammag   

                                                                                          NAME  \
0  data08_1beammag.00087764.physics_MBTS_BCM_LUCID.recon.HIST.o4_r605_tid032361                  
1  data08_1beammag.00088128.physics_RNDM.recon.HIST.o4_r605_tid032373                            
2  data08_1beammag.00087764.physics_MBTS_BCM_LUCID.recon.HIST.o4_r653_tid047010                  
3  data08_1beammag.00087764.physics_L1Calo.merge.HIST.o4_r653_p18_tid053115                      
4  data08_1beammag.00087764.physics_MBTS_BCM_LUCID.recon.NTUP_MUONCALIB.o4_r653_r792_tid082443   
5  data08_1beammag.00088128.physics_MBTS_BCM_LUCID.recon.NTUP_MUONCALIB.o4_r653_r792_tid082454   
6  data08_1beammag.00088069.physics_BPTX.daq.RAW.o4                                              
7  data08_1beammag.00088069.physics_BPTX.daq.RAW.o4                                              
8  data08_1beammag.00088069.physics_MBTS_BCM_LUCID.daq.RAW.o4                                    
9  data08_1beammag.00088105.physics_RNDM.daq.RAW.o4                                              

       CHILD_SCOPE  \
0  data08_1beammag   
1  data08_1beammag   
2  data08_1beammag   
3  data08_1beammag   
4  data08_1beammag   
5  data08_1beammag   
6  data08_1beammag   
7  data08_1beammag   
8  data08_1beammag   
9  data08_1beammag   

                                                         CHILD_NAME DID_TYPE  \
0  HIST.032361._00106.root.1                                         D         
1  HIST.032373._00007.root.1                                         D         
2  HIST.047010._00130.root.2                                         D         
3  HIST.053115._00001.root.1                                         D         
4  NTUP_MUONCALIB.082443._000122.root.1                              D         
5  NTUP_MUONCALIB.082454._000032.root.1                              D         
6  daq.ATLAS.0088069.physics.BPTX.LB0024.SFO-4._0011.data            D         
7  daq.ATLAS.0088069.physics.BPTX.LB0024.SFO-5._0005.data            D         
8  daq.ATLAS.0088069.physics.MBTS_BCM_LUCID.LB0017.SFO-5._0001.data  D         
9  daq.ATLAS.0088105.physics.RNDM.LB0001.SFO-3._0001.data            D         

  CHILD_TYPE LENGTH      BYTES   ADLER32   MD5 RULE_EVALUATION     UPDATED_AT  \
0  F          0      32531211   6f8569e6  None  0               1230596278000   
1  F          0      20145988   8c777d93  None  0               1230478143000   
2  F          0      28173163   56b56cee  None  0               1238688027000   
3  F          0      40480899   b08ffd4b  None  0               1239062123000   
4  F          0      8678520    3bf797ba  None  0               1253776096000   
5  F          0      1843905    6341d337  None  0               1253774998000   
6  F          0      360377120  42bb71ac  None  0               1221176343000   
7  F          0      360452020  818a71a6  None  0               1221176343000   
8  F          0      1440936    a2ef591a  None  0               1221176354000   
9  F          0      233783784  23ebd90e  None  0               1221180269000   

      CREATED_AT EVENTS  \
0  1230596278000  None    
1  1230478143000  None    
2  1238688027000  None    
3  1239062123000  None    
4  1253776096000  None    
5  1253774998000  None    
6  1221176343000  1000    
7  1221176343000  1000    
8  1221176354000  4       
9  1221180269000  16      

                                                                    join_key_lookup  \
0  data08_1beammag:HIST.032361._00106.root.1                                          
1  data08_1beammag:HIST.032373._00007.root.1                                          
2  data08_1beammag:HIST.047010._00130.root.2                                          
3  data08_1beammag:HIST.05311

In [15]:
df_lookup_dataset_count.count()

624370615

In [16]:
df_mai.count()

589283

In [17]:
df_lookup_dataset_count.createOrReplaceTempView("count")
df_lookup_final = spark.sql("SELECT *\
                             FROM mai LEFT JOIN count ON mai.join_key_mai == count.join_key_lookup_count")# AND count.distinct_datasets == 1")
df_lookup_final.count()

599278

In [18]:
df_lookup_final.createOrReplaceTempView("final")
spark.sql("SELECT * FROM final WHERE distinct_datasets IS NULL").limit(20).toPandas()

account  accountHash clientState  clientStateHash  \
0  ahasib  79838733     DONE       -878527175         

                                                                                                            dataset  \
0  mc16_13TeV.433206.ParticleGun_pid11_E2097152_disj_eta_m35_m30_30_35_zv_0.deriv.NTUP_FCS.e6648_s3281_r10283_p3474   

   datasetHash datasetScope  datasetScopeHash           dataset_version  \
0 -1436131747   mc16_13TeV  -1447489496        e6648_s3281_r10283_p3474   

   dataset_versionHash  datatype  datatypeHash eventType  eventTypeHash  \
0 -927544266            NTUP_FCS -1403378361    download  59378080        

  eventVersion  eventVersionHash                               filename  \
0  1.14.11     -2044360925        NTUP_FCS.13751310._000001.pool.root.1   

   filenameHash  filesize_double  hits           hostname  hostnameHash  \
0  1926376267    1.859035e+09     1     lxplus106.cern.ch -531576288      

  prod_step  prod_stepHash     project  projectHash protocol  protocolHash  \
0  deriv    -1058503065     mc16_13TeV -1447489496   srm      863299160      

                       remoteSite  remoteSiteHash run_number  run_numberHash  \
0  UKI-NORTHGRID-MAN-HEP_DATADISK  250770422       433206     1372444977       

    scopeHash                                             stream_name  \
0 -1447489496  ParticleGun_pid11_E2097152_disj_eta_m35_m30_30_35_zv_0   

   stream_nameHash                           traceId  traceIdHash  \
0 -917504542        2ead2de4c55e4f179ebc41bbb183d503 -501397853     

                traceIp  traceIpHash  traceTimeentryUnix  transferStart  \
0  ::ffff:137.138.31.92 -2083185428   1.527794e+09        1.527794e+09    

                               uuid   uuidHash  \
0  9a60a1664e9d4edd8d60f9977e79d8dd  417730461   

                                       join_key_mai   fileScope SCOPE  NAME  \
0  mc16_13TeV:NTUP_FCS.13751310._000001.pool.root.1  mc16_13TeV  None  None   

  CHILD_SCOPE CHILD_NAME DID_TYPE CHILD_TYPE LENGTH BYTES ADLER32   MD5  \
0  None        None       None     None       None   None  None    None   

  RULE_EVALUATION UPDATED_AT CREATED_AT EVENTS join_key_lookup  \
0  None            None       None       None   None             

  join_key_lookup_count distinct_datasets  
0  None                  None

## TODO: hier checken ob man manche distinct datasets im zweiten Teil wegwerfen könnte wie bei SCOPE == hc_test im april

In [27]:
spark.sql("SELECT * FROM final WHERE distinct_datasets > 1 ORDER BY distinct_datasets DESC").toPandas()

account  accountHash clientState  clientStateHash  \
0      bejaeger -479767920    DONE       -878527175         
1      bejaeger -479767920    DONE       -878527175         
2      bejaeger -479767920    DONE       -878527175         
3      bejaeger -479767920    DONE       -878527175         
4      bejaeger -479767920    DONE       -878527175         
5      bejaeger -479767920    DONE       -878527175         
6      bejaeger -479767920    DONE       -878527175         
7      bejaeger -479767920    DONE       -878527175         
8      bejaeger -479767920    DONE       -878527175         
9      bejaeger -479767920    DONE       -878527175         
10     bejaeger -479767920    DONE       -878527175         
11     bejaeger -479767920    DONE       -878527175         
12     bejaeger -479767920    DONE       -878527175         
13     bejaeger -479767920    DONE       -878527175         
14     bejaeger -479767920    DONE       -878527175         
15     bejaeger -479767920    DONE       -878527175         
16     bejaeger -479767920    DONE       -878527175         
17     bejaeger -479767920    DONE       -878527175         
18     bejaeger -479767920    DONE       -878527175         
19     bejaeger -479767920    DONE       -878527175         
20     bejaeger -479767920    DONE       -878527175         
21     bejaeger -479767920    DONE       -878527175         
22     bejaeger -479767920    DONE       -878527175         
23     bejaeger -479767920    DONE       -878527175         
24     bejaeger -479767920    DONE       -878527175         
25     bejaeger -479767920    DONE       -878527175         
26     bejaeger -479767920    DONE       -878527175         
27     bejaeger -479767920    DONE       -878527175         
28     bejaeger -479767920    DONE       -878527175         
29     bejaeger -479767920    DONE       -878527175         
...         ...        ...     ...              ...         
19960  bejaeger -479767920    DONE       -878527175         
19961  bejaeger -479767920    DONE       -878527175         
19962  bejaeger -479767920    DONE       -878527175         
19963  bejaeger -479767920    DONE       -878527175         
19964  bejaeger -479767920    DONE       -878527175         
19965  bejaeger -479767920    DONE       -878527175         
19966  bejaeger -479767920    DONE       -878527175         
19967  bejaeger -479767920    DONE       -878527175         
19968  bejaeger -479767920    DONE       -878527175         
19969  bejaeger -479767920    DONE       -878527175         
19970  mkbugge  -1662366609   DONE       -878527175         
19971  mkbugge  -1662366609   DONE       -878527175         
19972  mkbugge  -1662366609   DONE       -878527175         
19973  mkbugge  -1662366609   DONE       -878527175         
19974  tatsuya   1961187428   DONE       -878527175         
19975  tatsuya   1961187428   DONE       -878527175         
19976  tatsuya   1961187428   DONE       -878527175         
19977  tatsuya   1961187428   DONE       -878527175         
19978  nilang    159388291    DONE       -878527175         
19979  nilang    159388291    DONE       -878527175         
19980  fapetsch  56878527     DONE       -878527175         
19981  fapetsch  56878527     DONE       -878527175         
19982  fapetsch  56878527     DONE       -878527175         
19983  fapetsch  56878527     DONE       -878527175         
19984  hbaluchb  516336770    DONE       -878527175         
19985  hbaluchb  516336770    DONE       -878527175         
19986  kuno      467351034    DONE       -878527175         
19987  kuno      467351034    DONE       -878527175         
19988  jojungge -1078508397   DONE       -878527175         
19989  jojungge -1078508397   DONE       -878527175         

                                                                                                                                         dataset  \
0      data15_13TeV.00267638.physics_Main.deriv.DAOD_JETM5.r9264_p3083_p3518 

In [28]:
spark.sql("SELECT * FROM final WHERE SCOPE == 'hc_test' AND distinct_datasets > 1").toPandas()

Empty DataFrame
Columns: [account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_mai, fileScope, SCOPE, NAME, CHILD_SCOPE, CHILD_NAME, DID_TYPE, CHILD_TYPE, LENGTH, BYTES, ADLER32, MD5, RULE_EVALUATION, UPDATED_AT, CREATED_AT, EVENTS, join_key_lookup, join_key_lookup_count, distinct_datasets]
Index: []

In [20]:
df_lookup_final.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_lookup_final.columns]).limit(10).toPandas()

account  accountHash  clientState  clientStateHash  dataset  datasetHash  \
0  0        0            0            0                0        0             

   datasetScope  datasetScopeHash  dataset_version  dataset_versionHash  \
0  0             0                 0                0                     

   datatype  datatypeHash  eventType  eventTypeHash  eventVersion  \
0  0         0             0          0              0              

   eventVersionHash  filename  filenameHash  filesize_double  hits  hostname  \
0  0                 0         0             0                0     0          

   hostnameHash  prod_step  prod_stepHash  project  projectHash  protocol  \
0  0             0          0              0        0            0          

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0  0             0           0               0           0                

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0  0          0            0                0        0            0         

   traceIpHash  traceTimeentryUnix  transferStart  uuid  uuidHash  \
0  0            0                   0              0     0          

   join_key_mai  fileScope  SCOPE  NAME  CHILD_SCOPE  CHILD_NAME  DID_TYPE  \
0  0             0          1      1     1            1           1          

   CHILD_TYPE  LENGTH  BYTES  ADLER32     MD5  RULE_EVALUATION  UPDATED_AT  \
0  1           590858  1      1        599278  590856           1            

   CREATED_AT  EVENTS  join_key_lookup  join_key_lookup_count  \
0  1           69335   1                1                       

   distinct_datasets  
0  1

## check for multiple records

In [21]:
spark.sql("SELECT join_key_mai, COUNT(*)\
           FROM final \
           GROUP BY join_key_mai\
           ORDER BY 2 DESC").limit(20).toPandas()

join_key_mai  count(1)
0   mc16_13TeV:DAOD_JETM5.13995233._000240.pool.root.1  120     
1   mc16_13TeV:DAOD_JETM5.13995233._000241.pool.root.1  120     
2   mc16_13TeV:DAOD_JETM5.13995233._000238.pool.root.1  120     
3   mc16_13TeV:DAOD_JETM5.13995233._000242.pool.root.1  118     
4   mc16_13TeV:DAOD_JETM5.13995233._000244.pool.root.1  98      
5   mc16_13TeV:DAOD_JETM5.13995233._000243.pool.root.1  96      
6   mc16_13TeV:DAOD_JETM5.13995233._000246.pool.root.1  96      
7   mc16_13TeV:DAOD_JETM5.13995233._000245.pool.root.1  96      
8   mc16_13TeV:DAOD_JETM5.13995233._000239.pool.root.1  96      
9   mc16_13TeV:DAOD_JETM5.13995233._000247.pool.root.1  94      
10  mc16_13TeV:DAOD_JETM5.13995233._000258.pool.root.1  90      
11  mc16_13TeV:DAOD_JETM5.13995233._000248.pool.root.1  90      
12  mc16_13TeV:DAOD_JETM5.13995233._000255.pool.root.1  90      
13  mc16_13TeV:DAOD_JETM5.13995233._000251.pool.root.1  90      
14  mc16_13TeV:DAOD_JETM5.13995233._000254.pool.root.1  90      
15  mc16_13TeV:DAOD_JETM5.13995233._000250.pool.root.1  90      
16  mc16_13TeV:DAOD_JETM5.13995233._000260.pool.root.1  90      
17  mc16_13TeV:AOD.12911492._000017.pool.root.1         85      
18  mc16_13TeV:AOD.12911492._000016.pool.root.1         84      
19  mc16_13TeV:AOD.12911492._000019.pool.root.1         76

In [22]:
temptest = spark.sql("SELECT join_key_mai, COUNT(*)\
           FROM final \
           GROUP BY join_key_mai\
           ORDER BY 2 DESC")
temptest.where("join_key_mai == 'mc16_13TeV:NTUP_PILEUP.13658010._000008.pool.root.1'").toPandas()

join_key_mai  count(1)
0  mc16_13TeV:NTUP_PILEUP.13658010._000008.pool.root.1  2

In [23]:
spark.sql("SELECT *\
           FROM final \
           WHERE join_key_mai == 'mc16_5TeV:DAOD_EGAM1.13695903._000004.pool.root.1'").limit(20).toPandas()

Empty DataFrame
Columns: [account, accountHash, clientState, clientStateHash, dataset, datasetHash, datasetScope, datasetScopeHash, dataset_version, dataset_versionHash, datatype, datatypeHash, eventType, eventTypeHash, eventVersion, eventVersionHash, filename, filenameHash, filesize_double, hits, hostname, hostnameHash, prod_step, prod_stepHash, project, projectHash, protocol, protocolHash, remoteSite, remoteSiteHash, run_number, run_numberHash, scopeHash, stream_name, stream_nameHash, traceId, traceIdHash, traceIp, traceIpHash, traceTimeentryUnix, transferStart, uuid, uuidHash, join_key_mai, fileScope, SCOPE, NAME, CHILD_SCOPE, CHILD_NAME, DID_TYPE, CHILD_TYPE, LENGTH, BYTES, ADLER32, MD5, RULE_EVALUATION, UPDATED_AT, CREATED_AT, EVENTS, join_key_lookup, join_key_lookup_count, distinct_datasets]
Index: []

In [24]:
df_lookup_final.describe().toPandas()

summary   account           accountHash         clientState  \
0  count   599278    599278                599278               
1  mean    None      -8.461948508828624E7  None                 
2  stddev  None      1.1243370814962916E9  None                 
3  min     abell     -2123889579           CannotAuthenticate   
4  max     zmarshal  2140382084            SourceNotFound       

         clientStateHash  \
0  599278                  
1  -1.0627545960871098E9   
2  2.749591578691901E8     
3  -1913538301             
4  634051254               

                                                                                           dataset  \
0  599278                                                                                            
1  None                                                                                              
2  None                                                                                              
3  data12_8TeV.00200805.physics_MinBias.recon.NTUP_MINBIAS.r4768                                     
4  mc16_valid.361034.Pythia8EvtGen_A2MSTW2008LO_minbias_inelastic_low.merge.HITS.e3581_s3028_s2939   

             datasetHash datasetScope      datasetScopeHash dataset_version  \
0  599278                 599278       599278                599278           
1  -1.2178728948616501E7  None         -9.374355000650616E8  None             
2  1.2693128681762025E9   None         6.760454481990764E8   None             
3  -2147401420            data12_8TeV  -2057006107           c1128_m1842      
4  2147445885             mc16_valid   2109319881            x563_m1972       

    dataset_versionHash datatype          datatypeHash eventType  \
0  599278                599278   599278                599278     
1  -3.521113310391504E7  None     2.207480624584383E8   None       
2  1.0974504113502483E9  None     1.0676911195816748E9  None       
3  -2144917680           AOD      -2070622869           download   
4  2146973161            log      2124512252            download   

  eventTypeHash eventVersion       eventVersionHash  \
0  599278        599278       599278                  
1  5.937808E7    None         -2.0120598721819656E9   
2  0.0           None         3.139418720827351E8     
3  59378080      1.14.10      -2044360925             
4  59378080      1.16.1       1261702107              

                             filename          filenameHash  \
0  599278                              599278                 
1  None                                398096.5534643354      
2  None                                1.2390972686014647E9   
3  AOD.05341490._000446.pool.root.1    -2147471303            
4  log.14217468._003414.job.log.tgz.1  2147478625             

        filesize_double               hits                  hostname  \
0  599278                599278             599278                     
1  1.4366080142069173E9  4.453669248662557  None                       
2  2.220584050647932E9   6.29523465299338   None                       
3  9576.0                1                  ReCaS-ui-01.cs.infn.it     
4  1.1976018886E10       85                 xenia.nevis.columbia.edu   

           hostnameHash prod_step          prod_stepHash      project  \
0  599278                599278    599278                 599278        
1  -4.678854145521444E7  None      -2.6585467887347442E8  None          
2  9.710591582793208E8   None      9.240392891635021E8    None          
3  -2138306752           PhysCont  -1058503065            data12_8TeV   
4  2129595856            simul     2037678380             mc16_valid    

            projectHash protocol         protocolHash  \
0  599278                599278   599278                
1  -9.374355000650616E8  None     4.492775039871796E8   
2  6.760454481990764E8   None     1.102129994765002E9   
3  -2057006107           davs     -1607963002           
4  2109319881            srm      2128034789            

                  re

In [29]:
df_lookup_final.write.format("json").save("/user/sprenner/lookup_table_1_official_mai.json")